In [1]:
from datetime import datetime
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
import os
import statistics

%matplotlib inline

In [ ]:
contract_loc = 'data/sc_vol_adj_futures/'
contract_files = list(file for file in os.listdir(contract_loc) if file[-3:] == 'csv')

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
Run this every weekday at 13:00 GMT+7
"""
data_futures = {}
for file in contract_files:
    data_futures['-'.join((file.split('.')[0].split('-')[0][:-3], file.split('.')[0].split('-')[-1]))] = pd.read_csv(f'{contract_loc}/{file}', usecols=[0, 1], index_col=0, parse_dates=True)

In [4]:
def volatility(ts, lookback=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(lookback).std().iloc[-1]

def ewm_volatility(ts, lookback=36):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change using ewm calculation
    """
    return ts.pct_change().ewm(span=lookback).std().iloc[-1]

In [5]:
# Create an empty DataFrame to store score
ins_risk_table = pd.DataFrame(columns=['ticker', 'last_date', 'ins_risk_starter', 'ins_risk_staunch'])

# How many (series) candles back for std dev calculation?
vola_window = 24

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_futures.items():
    ins_risk_starter = volatility(timeseries[' Last']) * 16
    ins_risk_staunch = ewm_volatility(timeseries[' Last']) * 16
    last_date = timeseries.index[-1]
    ins_risk_table = ins_risk_table.append({'ticker': ticker,
                                            'last_date': last_date,
                                            'ins_risk_starter': ins_risk_starter,
                                            'ins_risk_staunch': ins_risk_staunch},
                                           ignore_index=True)

In [6]:
ins_risk_table.to_clipboard()
ins_risk_table

,ticker,last_date,ins_risk_starter,ins_risk_staunch
0,FXXE-EUREX,2020-10-15,0.200505,0.193141
1,RY-CME,2020-10-15,0.067479,0.066742
2,OJ-OJX20,2020-10-15,0.281417,0.274216
3,HO-NYMEX,2020-10-15,0.400942,0.397789
4,EMD-CME,2020-10-15,0.207700,0.199769
...,...,...,...,...
60,ZB-CBOT,2020-10-15,0.051864,0.063727
61,FXXP-EUREX,2020-10-15,0.182453,0.177401
62,6E-CME,2020-10-15,0.062019,0.061799
63,ZF-CBOT,2020-10-15,0.009597,0.011882
